In [0]:
bronze_output = dbutils.jobs.taskValues.get(taskKey = 'Bronze', key = 'bronze_output')

start_date = bronze_output.get('start_date','')
bronze_adls = bronze_output.get('bronze_adls','')
silver_adls = bronze_output.get('bronze_adls','')

print(f"Start Date: {start_date}, Bronze ADLS:{bronze_adls}")
      


In [0]:

from pyspark.sql.functions import col, isnull, when
from pyspark.sql.types import TimestampType
from datetime import date, timedelta

In [0]:
# Load the JSON data into a Spark DataFrame
df = spark.read.option("multiline", "true").json(f"{bronze_adls}{start_date}_earthquake_data.json")
     

In [0]:
df.head()

Row(geometry=Row(coordinates=[-122.810997009277, 38.839168548584, 1.75999999046326], type='Point'), id='nc75231007', properties=Row(alert=None, cdi=None, code='75231007', detail='https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc75231007&format=geojson', dmin=0.01577, felt=None, gap=124.0, ids=',nc75231007,', mag=0.69, magType='md', mmi=None, net='nc', nst=7, place='8 km WNW of Cobb, CA', rms=0.0, sig=7, sources=',nc,', status='automatic', time=1756598036030, title='M 0.7 - 8 km WNW of Cobb, CA', tsunami=0, type='earthquake', types=',nearby-cities,origin,phase-data,', tz=None, updated=1756598133772, url='https://earthquake.usgs.gov/earthquakes/eventpage/nc75231007'), type='Feature')

In [0]:
# Reshape earthquake data
df = (
    df
    .select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
    )
)

In [0]:
df.head()

Row(id='nc75231007', longitude=-122.810997009277, latitude=38.839168548584, elevation=1.75999999046326, title='M 0.7 - 8 km WNW of Cobb, CA', place_description='8 km WNW of Cobb, CA', sig=7, mag=0.69, magType='md', time=1756598036030, updated=1756598133772)

In [0]:
# Validate data
df = (
    df
    .withColumn('longitude', when(isnull(col('longitude')), 0).otherwise(col('longitude')))
    .withColumn('latitude', when(isnull(col('latitude')), 0).otherwise(col('latitude')))
    .withColumn('time', when(isnull(col('time')), 0).otherwise(col('time')))
)

In [0]:
# Convert 'time' and 'updated' to timestamp from Unix time
df = (
    df
    .withColumn('time', (col('time') / 1000).cast(TimestampType()))
    .withColumn('updated', (col('updated') / 1000).cast(TimestampType()))
)

In [0]:
df.head()

Row(id='nc75231007', longitude=-122.810997009277, latitude=38.839168548584, elevation=1.75999999046326, title='M 0.7 - 8 km WNW of Cobb, CA', place_description='8 km WNW of Cobb, CA', sig=7, mag=0.69, magType='md', time=datetime.datetime(2025, 8, 30, 23, 53, 56, 30000), updated=datetime.datetime(2025, 8, 30, 23, 55, 33, 772000))

In [0]:
# Save the transformed DataFrame to the Silver container
silver_output_path = f"{silver_adls}earthquake_events_silver/"

In [0]:
# Append DataFrame to Silver container in Parquet format
df.write.mode('append').parquet(silver_output_path)

In [0]:
dbutils.jobs.taskValues.set(key = "silver_output", value = silver_output_path)